# Gallery Example: Multi-Class Closed Queueing Network

This example demonstrates a multi-class closed queueing network:
- **Classes**: Multiple job classes (default 2)
- **Stations**: Configurable number of queues (default 1)
- **Topology**: Serial routing with optional delay station
- **Population**: 5 jobs per class
- **Scheduling**: Processor Sharing (PS)
- **Service**: Random exponential service rates per class/station

This model is useful for analyzing systems with different customer types or job characteristics.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_cqn_multiclass(m=1, r=2, wantdelay=true, seed=None): Network {    """Create multi-class closed queueing network        Parameters:    - m: Number of queue stations (default 1)    - r: Number of classes (default 2)    - wantdelay: Whether to include delay station (default true)    - seed: Random seed for reproducibility    """    if seed is not None:        random.seed(seed)        np.random.seed(seed)        model = Network("Multi-class CQN")        # Block 1: nodes    node = []    for i in range(m):        queue = Queue(model, f"Queue {i+1}", SchedStrategy.PS)        node.append(queue)        if wantdelay:        delay = Delay(model, "Delay 1")        node.append(delay)        # Block 2: classes    jobclass = []    for s in range(r):        jclass = ClosedClass(model, f"Class{s+1}", 5, node[0], 0)        jobclass.append(jclass)        # Set service rates    service_rates = {}  # Store for analysis    for s in range(r):        service_rates[f"Class{s+1}"] = {}        for i in range(m):            rate = round(50 * random.random())  # Random rate 0-50            if rate == 0:                rate = 1  # Avoid zero service rate            node[i].setService(jobclass[s], Exp.fitMean(rate))            service_rates[f"Class{s+1}"][f"Queue{i+1}"] = rate                if wantdelay:            delay_rate = round(100 * random.random())  # Random delay 0-100            if delay_rate == 0:                delay_rate = 1            node[-1].setService(jobclass[s], Exp.fitMean(delay_rate))            service_rates[f"Class{s+1}"]["Delay1"] = delay_rate        # Block 3: topology - serial routing for each class    P = model.initRoutingMatrix()    for s in range(r):        # Create cyclic routing for this class        for i in range(len(node)):            next_node = node[(i + 1) % len(node)]            P.addRoute(jobclass[s], node[i], next_node, 1.0)        model.link(P)        return model, service_rates# Create the model with default parametersmodel, service_rates = gallery_cqn_multiclass(m=1, r=2, wantdelay=true, seed=42)println(f"Population: 5 jobs per class")println(f"\nService rates (mean service times):")for class_name, rates in service_rates.items():    println(f"  {class_name}: {rates}")

## Multi-Class Network Analysis

In multi-class closed networks:
- **Classes compete** for server resources
- **Different service requirements** per class/station
- **Population constraints** limit total jobs per class
- **PS scheduling** provides fair sharing among classes

Performance depends on:
- Population mix and sizes
- Service rate differences between classes
- Network topology and routing

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver (exact for closed networks)
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// NC Solver
val solver_nc = NC(model)
val avg_table_nc = solver_nc.avgTable
println("\nNC Solver:")
println(avg_table_nc)

In [ ]:
// Analyze class-specific performance
println("\n=== Class-Specific Performance Analysis ===")
// Get detailed performance metrics by class
val solver = MVA(model)
val avg_table = solver.avgTable

println("Per-class throughput and response times:")
for class_idx in range(model.get_number_of_classes()):
    val class_name = f"Class{class_idx + 1}"
    println(f"\n{class_name}:")
// Calculate total response time and throughput for this class
    val total_resp_time = 0
    for station_idx in range(1, model.get_number_of_stations()):
        try:
// Note: Table structure may vary - this is a general approach
            val resp_time = float(avg_table.iloc[station_idx, 2])  # Response time column
            total_resp_time += resp_time
        except:
            pass
    
    println(f"  Service rates: {service_rates[class_name]}")
    println(f"  Population: 5 jobs")

In [ ]:
// Compare different class population mixes
println("\n=== Population Mix Analysis ===")

fun create_cqn_with_populations(populations): Network {
    """Create CQN with specified populations per class"""
    val model_pop = Network("CQN-PopMix")
// Create nodes
    val queue = Queue(model_pop, "Queue1", SchedStrategy.PS)
    val delay = Delay(model_pop, "Delay1")
    val nodes = [queue, delay]
// Create classes with different populations
    val jobclasses = []
    for i, pop in enumerate(populations):
        val jclass = ClosedClass(model_pop, f"Class{i+1}", pop, queue, 0)
        jobclasses.append(jclass)
// Set service rates (fixed for comparison)
        queue.setService(jclass, Exp.fitMean(10 + i * 5))  # Different service rates
        delay.setService(jclass, Exp.fitMean(20 + i * 10))
// Routing
    val P = model_pop.initRoutingMatrix()
    for jclass in jobclasses:
        P.addRoute(jclass, queue, delay, 1.0)
        P.addRoute(jclass, delay, queue, 1.0)
    
    model_pop.link(P)
    return model_pop
// Test different population mixes
val population_scenarios = [
    ([5, 5], "Balanced"),
    ([10, 2], "Class1 Heavy"),
    ([2, 10], "Class2 Heavy"),
    ([8, 8], "High Load")
]

println("Population Mix | Total Pop | System Performance")
println("-" * 50)

for populations, scenario_name in population_scenarios:
    try:
        val model_scenario = create_cqn_with_populations(populations)
        val solver_scenario = MVA(model_scenario)
        val avg_table_scenario = solver_scenario.avgTable
        
        val total_pop = sum(populations)
// Calculate system utilization (approximate)
        val util_estimate = float(avg_table_scenario.iloc[1, 1]) if len(avg_table_scenario) > 1 else 0
        
        println(f"{populations} | {total_pop:8d}  | Util≈{util_estimate:.3f} ({scenario_name})")
    except Exception as e:
        println(f"{populations} | {sum(populations):8d}  | Error: {str(e)[:20]}...")

In [ ]:
// Analyze scaling with number of classes
println("\n=== Multi-Class Scaling Analysis ===")

fun create_cqn_r_classes(num_classes, population_per_class=3): Network {
    """Create CQN with specified number of classes"""
    val model_r = Network(f"CQN-{num_classes}Classes")
// Single queue + delay
    val queue = Queue(model_r, "Queue", SchedStrategy.PS)
    val delay = Delay(model_r, "Delay")
    val nodes = [queue, delay]
// Create classes
    val jobclasses = []
    for c in range(num_classes):
        val jclass = ClosedClass(model_r, f"C{c+1}", population_per_class, queue, 0)
        jobclasses.append(jclass)
// Differentiated service rates
        val queue_rate = 5 + c * 2  # Increasing service requirements
        val delay_rate = 15 + c * 3
        
        queue.setService(jclass, Exp.fitMean(queue_rate))
        delay.setService(jclass, Exp.fitMean(delay_rate))
// Serial routing for all classes
    val P = model_r.initRoutingMatrix()
    for jclass in jobclasses:
        P.addRoute(jclass, queue, delay, 1.0)
        P.addRoute(jclass, delay, queue, 1.0)
    
    model_r.link(P)
    return model_r
// Test different numbers of classes
val class_counts = [1, 2, 3, 4, 5]

println("Classes | Total Jobs | Queue Utilization | System Response")
println("-" * 60)

for num_classes in class_counts:
    try:
        val model_classes = create_cqn_r_classes(num_classes, population_per_class=3)
        val solver_classes = MVA(model_classes)
        val avg_table_classes = solver_classes.avgTable
        
        val total_jobs = num_classes * 3
        val queue_util = float(avg_table_classes.iloc[1, 1]) if len(avg_table_classes) > 1 else 0
        
        println(f"   {num_classes:2d}   |     {total_jobs:2d}     |      {queue_util:.3f}       | Complex system")
    except Exception as e:
        println(f"   {num_classes:2d}   |     {num_classes*3:2d}     | Error: analysis failed")

println("\nNote: As the number of classes increases, resource contention and")
println("scheduling complexity grow, affecting overall system performance.")